#Directory Path

In [ ]:
from google.colab import drive
import os 

from os import listdir
from os.path import isfile, join


input_music_disco='/content/drive/My Drive/DeepLearning_2021/proyecto/Data/disco_wav'
input_music_pop = '/content/drive/My Drive/DeepLearning_2021/proyecto/Data/pop_wav' 
save_disco ='/content/drive/My Drive/DeepLearning_2021/proyecto/Data/New_input_disco'
save_pop = '/content/drive/My Drive/DeepLearning_2021/proyecto/Data/New_input_pop'

filepath = input_music_disco # or input_music_pop
onlyfiles = [f for f in listdir(filepath) if isfile(join(filepath, f))]

# Funtions to create the images from waves.
We used GTZAN dataset

In [ ]:
import librosa
import numpy
import skimage.io

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def spectrogram_image(y, sr, out, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
                                            n_fft=hop_length*2, hop_length=hop_length)
    mels = numpy.log(mels + 1e-9) # add small number to avoid log(0)
   

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(numpy.uint8)

    img = numpy.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy
  
    # save as PNG
    skimage.io.imsave(out, img)

In [ ]:
# settings
hop_length = 512 # number of samples per time-step in spectrogram
n_mels = 128 # number of bins in spectrogram. Height of image
time_steps = 127 # number of time-steps. Width of image



for i in range(0,100):
  input_filename = filepath+'/'+str(onlyfiles[i])
  # load audio. Using example from librosa
  y, sr = librosa.load(input_filename, offset=1.0, duration=10.0, sr=22050)
  out = save_disco + str(i)+'.png'

  # extract a fixed length window
  start_sample = 0 # starting at beginning
  length_samples = time_steps*hop_length
  window = y[start_sample:start_sample+length_samples]

  # convert to PNG
  spectrogram_image(window, sr=sr, out=out, hop_length=hop_length, n_mels=n_mels)